In [22]:
pip install gmplot


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.7/164.7 kB 3.7 MB/s eta 0:00:00


In [ ]:
pip install cartopy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 17.6 MB/s eta 0:00:00


In [ ]:
pip install basemap

In [ ]:
import pandas as pd

In [ ]:
file_135='/content/df_135_labeled_on_lift.csv'
df_135=pd.read_csv(file_135)

In [ ]:
df_135.columns

In [ ]:
df_135.info()

In [ ]:
df_135['ActivityConfidence'].value_counts

In [ ]:
df_135['ActivityConfidence'].info()

In [ ]:
df_135['ActivityConfidence'].describe()

In [ ]:
import matplotlib.pyplot as plt

plt.hist(df_135['ActivityConfidence'], bins=20)
plt.xlabel('Activity Confidence')
plt.ylabel('Frequency')
plt.title('Histogram of Activity Confidence')
plt.show()

In [ ]:
df_135['ActivityType'].describe()

In [ ]:
df_135['ActivityType'].value_counts()

In [ ]:
import matplotlib.pyplot as plt

plt.hist(df_135['ActivityType'], bins=20)
plt.xlabel('ActivityType')
plt.ylabel('Frequency')
plt.title('Histogram of ActivityType')
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Plot data points with Unknown activity type
unknown_data = df_135[df_135['ActivityType'] == 'Unknown']
plt.scatter(unknown_data['Long'], unknown_data['Lat'], c='red', label='Unknown')

# Plot data points with other activity types
known_data = df_135[df_135['ActivityType'] != 'Unknown']
plt.scatter(known_data['Long'], known_data['Lat'], c='blue', label='Known')

plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('GPS Data Points')
plt.legend()
plt.show()

In [ ]:
pip install folium

In [ ]:
file_AGG='/content/Andermatt_Gondelbahn_Gutsch-2024-02-04_11-58-32.csv'
df_AGG=pd.read_csv(file_AGG)

In [ ]:
df_AGG.columns


In [ ]:
df_AGG.info()

In [ ]:
import plotly.express as px
# Create the scatter plot using Plotly Express
fig = px.scatter(df_AGG,
                 x=df_AGG['Timestamp'],
                 y=df_AGG['Alt(m)'])

# Update the layout with title and axis labels
fig.update_layout(
        title='Altitude over Time',
        xaxis_title='Timestamp',
        yaxis_title='Altitude (m)',
        coloraxis_colorbar=dict(title='Cluster Label')
    )

# Show the Plotly figure
fig.show()

In [ ]:
#Add 'on_lift' column to df and set all values to 0
df_AGG['on_lift'] = 0

#Manually noted timestamps for lift rides
lift_rides = [
    ('2024-02-04 11:58:36', '2024-02-04 12:01:59'),
    ('2024-02-04 12:03:29', '2024-02-04 12:10:11'),
    ('2024-02-04 12:54:05', '2024-02-04 13:01:11'),
    ('2024-02-04 13:27:37', '2024-02-04 13:30:13'),
    ('2024-02-04 13:50:25', '2024-02-04 13:54:09'),
    ('2024-02-04 14:04:31', '2024-02-04 14:10:44'),
    ('2024-02-04 14:40:18', '2024-02-04 14:43:41'),
    ('2024-02-04 15:39:07', '2024-02-04 15:43:41'),
]
df_AGG['Timestamp'] = pd.to_datetime(df_AGG['Timestamp'])
#Update 'on_lift' column with lift_rides list
for start, end in lift_rides:
    start_time = pd.to_datetime(start)
    end_time = pd.to_datetime(end)
    df_AGG.loc[(df_AGG['Timestamp'] >= start_time) & (df_AGG['Timestamp'] <= end_time), 'on_lift'] = 1

In [ ]:
#def plot_results()
    # Plot for Altitude over Time, colored by 'on_lift' status
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 5))

    # Plot points where 'on_lift' is 1 with one color
plt.scatter(df_AGG.loc[df_AGG['on_lift'] == 1, 'Timestamp'],
                df_AGG.loc[df_AGG['on_lift'] == 1, 'Alt(m)'],
                c='red',
                label='On the lift',
                marker='o')

    # Plot points where 'on_lift' is 0 with another color
plt.scatter(df_AGG.loc[df_AGG['on_lift'] == 0, 'Timestamp'],
                df_AGG.loc[df_AGG['on_lift'] == 0, 'Alt(m)'],
                c='blue',
                label='Not on the lift',
                marker='o')

plt.title('Andermatt_Gondelbahn_Gutsch')
plt.xlabel('Timestamp')
plt.ylabel('Altitude (m)')
plt.legend()
plt.show()

In [ ]:
import folium

def generate_map(df):
    # Create a map centered on the mean latitude and longitude
    map_center = [df['Lat'].mean(), df['Long'].mean()]
    mymap = folium.Map(location=map_center, zoom_start=10)

    # Plot data points with on_lift type
    on_lift = df[df['on_lift'] == 0]
    for _, row in on_lift.iterrows():
        folium.CircleMarker(location=[row['Lat'], row['Long']], radius=5, color='red', fill=True, fill_color='red', tooltip=str(row['Timestamp'])).add_to(mymap)

    # Plot data points with not_on_lift  types
    not_on_lift = df[df['on_lift'] != 0]
    for _, row in not_on_lift.iterrows():
        folium.CircleMarker(location=[row['Lat'], row['Long']], radius=5, color='blue', fill=True, fill_color='blue', tooltip=str(row['Timestamp'])).add_to(mymap)

    # Return the map object
    return mymap

# Call the function with your DataFrame
mymap = generate_map(df_AGG)
mymap

In [ ]:
import folium
from folium import plugins

# Create a map centered on the mean latitude and longitude
map_center = [df_135['Lat'].mean(), df_135['Long'].mean()]
mymap = folium.Map(location=map_center, zoom_start=10)

# Plot data points with on_lift  type
on_lift = df_135[df_135['on_lift'] == 0]
for _, row in on_lift.iterrows():
    folium.CircleMarker(location=[row['Lat'], row['Long']], radius=5, color='red', fill=True, fill_color='red').add_to(mymap)

# Plot data points with not_on_lift types
not_on_lift = df_135[df_135['on_lift'] == 1]
for _, row in not_on_lift.iterrows():
    folium.CircleMarker(location=[row['Lat'], row['Long']], radius=5, color='blue', fill=True, fill_color='blue').add_to(mymap)

# Display the map in the notebook
mymap

In [ ]:
import folium
from folium import plugins

# Create a map centered on the mean latitude and longitude
map_center = [df_135['Lat'].mean(), df_135['Long'].mean()]
mymap = folium.Map(location=map_center, zoom_start=10)

# Plot data points with on_lift type
on_lift = df_135[df_135['on_lift'] == 0]
for _, row in on_lift.iterrows():
    folium.CircleMarker(location=[row['Lat'], row['Long']], radius=5, color='red', fill=True, fill_color='red', tooltip=str(row['Timestamp'])).add_to(mymap)

# Plot data points with not on_lift types
not_on_lift = df_135[df_135['on_lift'] != 0]
for _, row in not_on_lift.iterrows():
    folium.CircleMarker(location=[row['Lat'], row['Long']], radius=2, color='blue', fill=True, fill_color='blue', tooltip=str(row['Timestamp'])).add_to(mymap)

# Display the map in the notebook
mymap

In [ ]:
file_166='/content/df_166m_labeled_on_lift.csv.csv'
df_166=pd.read_csv(file_166)

In [ ]:
import matplotlib.pyplot as plt

# Plot data points with Unknown activity type
unknown_data = df_166[df_166['ActivityType'] == 'Unknown']
plt.scatter(unknown_data['Long'], unknown_data['Lat'], c='red', label='Unknown')

# Plot data points with other activity types
known_data = df_166[df_166['ActivityType'] != 'Unknown']
plt.scatter(known_data['Long'], known_data['Lat'], c='blue', label='Known')

plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('GPS Data Points')
plt.legend()
plt.show()

In [ ]:
import folium
from folium import plugins

# Create a map centered on the mean latitude and longitude
map_center = [df_166['Lat'].mean(), df_166['Long'].mean()]
mymap = folium.Map(location=map_center, zoom_start=10)

# Plot data points with on_lift type
on_lift = df_166[df_166['on_lift'] == 0]
for _, row in on_lift.iterrows():
    folium.CircleMarker(location=[row['Lat'], row['Long']], radius=5, color='red', fill=True, fill_color='red').add_to(mymap)

# Plot data points with not_on_lift types
not_on_lift = df_166[df_166['on_lift'] != 0]
for _, row in not_on_lift.iterrows():
    folium.CircleMarker(location=[row['Lat'], row['Long']], radius=5, color='blue', fill=True, fill_color='blue').add_to(mymap)

# Display the map in the notebook
mymap

In [ ]:
import folium

def generate_map(df):
    # Create a map centered on the mean latitude and longitude
    map_center = [df['Lat'].mean(), df['Long'].mean()]
    mymap = folium.Map(location=map_center, zoom_start=10)

    # Plot data points with on_lift type
    on_lift = df[df['on_lift'] == 0]
    for _, row in on_lift.iterrows():
        folium.CircleMarker(location=[row['Lat'], row['Long']], radius=5, color='red', fill=True, fill_color='red', tooltip=str(row['Timestamp'])).add_to(mymap)

    # Plot data points with not_on_lift  types
    not_on_lift = df[df['on_lift'] != 0]
    for _, row in not_on_lift.iterrows():
        folium.CircleMarker(location=[row['Lat'], row['Long']], radius=5, color='blue', fill=True, fill_color='blue', tooltip=str(row['Timestamp'])).add_to(mymap)

    # Return the map object
    return mymap

# Call the function with your DataFrame
mymap = generate_map(df_135)
mymap

In [ ]:
import folium

def generate_map(df):
    # Create a map centered on the mean latitude and longitude
    map_center = df['Lat'].mean(), df['Long'].mean()
    mymap = folium.Map(location=map_center, zoom_start=10)

    # Plot data points with on_lift type
    on_lift = df[df['on_lift'] == 0]
    for _, row in on_lift.iterrows():
        folium.CircleMarker(location=[row['Lat'], row['Long']], radius=8, color='red', fill=True, fill_color='red', tooltip=str(row['Timestamp'])).add_to(mymap)

    # Plot data points with not_on_lift  types
    not_on_lift = df[df['on_lift'] != 0]
    for _, row in not_on_lift.iterrows():
        folium.CircleMarker(location=[row['Lat'], row['Long']], radius=4, color='blue', fill=True, fill_color='blue', tooltip=str(row['Timestamp'])).add_to(mymap)

    # Return the map object
    return mymap

# Call the function with your DataFrame
mymap = generate_map(df_135)
mymap

In [ ]:
file_path='/content/v5_20240203_091405_135m.csv'
df_135_csv=pd.read_csv(file_path)

In [ ]:
import folium

def generate_map(df):
    # Create a map centered on the mean latitude and longitude
    map_center = df['Lat'].mean(), df['Long'].mean()
    mymap = folium.Map(location=map_center, zoom_start=10)

    # Plot data points with on_lift type
    on_lift = df[df['on_lift'] == 0]
    for _, row in on_lift.iterrows():
        folium.CircleMarker(location=[row['Lat'], row['Long']], radius=8, color='red', fill=True, fill_color='red', tooltip=str(row['Timestamp'])).add_to(mymap)

    # Plot data points with not_on_lift  types
    not_on_lift = df[df['on_lift'] != 0]
    for _, row in not_on_lift.iterrows():
        folium.CircleMarker(location=[row['Lat'], row['Long']], radius=4, color='blue', fill=True, fill_color='blue', tooltip=str(row['Timestamp'])).add_to(mymap)

    # Return the map object
    return mymap

# Call the function with your DataFrame
mymap = generate_map(df_135_csv)
mymap

In [ ]:
df_135.columns


In [ ]:
import folium

def generate_map(df, predictions=None):
    # Create a map centered on the mean latitude and longitude
    map_center = [df['Lat'].mean(), df['Long'].mean()]
    mymap = folium.Map(location=map_center, zoom_start=10)

    # Plot data points with on_lift type
    on_lift = df[df['on_lift'] == 0]
    for _, row in on_lift.iterrows():
        tooltip = str(row['Timestamp'])
        if predictions is not None and row['Timestamp'] in predictions:
            tooltip += f", Prediction: {predictions[row['Timestamp']]}"
        folium.CircleMarker(location=[row['Lat'], row['Long']], radius=5, color='red', fill=True, fill_color='red', tooltip=tooltip).add_to(mymap)

    # Plot data points with not_on_lift types
    not_on_lift = df[df['on_lift'] != 0]
    for _, row in not_on_lift.iterrows():
        tooltip = str(row['Timestamp'])
        if predictions is not None and row['Timestamp'] in predictions:
            tooltip += f", Prediction: {predictions[row['Timestamp']]}"
        folium.CircleMarker(location=[row['Lat'], row['Long']], radius=5, color='blue', fill=True, fill_color='blue', tooltip=tooltip).add_to(mymap)

    # Return the map object
    return mymap

# Define predictions dictionary
predictions = {}

# Iterate over the DataFrame and extract unique identifiers
for index, row in df_135.iterrows():
    # Assuming 'Timestamp' is the column name containing unique identifiers
    unique_id = row['Timestamp']

    # Add the unique ID to the predictions dictionary with a placeholder prediction
    predictions[unique_id] = 'Placeholder_Prediction'

# Call the generate_map function with your DataFrame and predictions
mymap = generate_map(df_135, predictions)

# Display the map
mymap

In [ ]:
import gmplot

# Initialize the Google Maps plot centered at the mean latitude and longitude
gmap = gmplot.GoogleMapPlotter(df_135['Lat'].mean(), df_135['Long'].mean(), 10)

# Plot data points with Unknown activity type
unknown_data = df_135[df_135['ActivityType'] == 'Unknown']
gmap.scatter(unknown_data['Lat'], unknown_data['Long'], 'red', size=40, marker=False)

# Plot data points with other activity types
known_data = df_135[df_135['ActivityType'] != 'Unknown']
gmap.scatter(known_data['Lat'], known_data['Long'], 'blue', size=40, marker=False)

# Draw the map to an HTML file
gmap.draw("map.html")

In [ ]:
gmap

In [ ]:
import webbrowser

# Specify the path to the HTML file
html_file_path = "map.html"

# Open the HTML file in the default web browser
webbrowser.open(html_file_path)

In [ ]:


import webbrowser

# Specify the path to the HTML file
html_file_path = "map.html"

# Open the HTML file in the default web browser
success = webbrowser.open(html_file_path)

if success:
    print("Web browser opened successfully.")
else:
    print("Failed to open web browser.")

In [ ]:
from IPython.display import IFrame

# Specify the path to the HTML file generated by Folium
html_file_path = "map.html"

# Display the HTML file in the notebook
IFrame(src=html_file_path, width=700, height=600)

In [ ]:
# Convert 'ActivityConfidence' column to float, ignoring errors
#df_135['ActivityConfidence'] = pd.to_numeric(df_135['ActivityConfidence'], errors='coerce')

In [ ]:
#df_135.info()

In [ ]:
#df_135['ActivityConfidence'].value_counts

In [ ]:
# Convert 'ActivityConfidence' column to float, replacing NaN with 0
#df_135['ActivityConfidence'] = pd.to_numeric(df_135['ActivityConfidence'], errors='coerce').fillna(0)

In [ ]:
df_135['ActivityConfidence'].value_counts

In [ ]:
#df_135 = df_135.drop(columns=['ActivityConfidence'])

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Instantiate LabelEncoder
label_encoder = LabelEncoder()

# Encode the target variable (ActivityType) into numerical values
y_test_encoded = label_encoder.fit_transform(y_test)

# Now, evaluate the performance of the trained model on the test set
y_pred = clf.predict(X_test_imputed)
print(classification_report(y_test_encoded, y_pred))

In [ ]:
from sklearn.impute import SimpleImputer

# Instantiate the imputer with strategy 'mean', 'median', or 'most_frequent'
imputer = SimpleImputer(strategy='mean')

# Fit the imputer to the training data and transform the data
X_train_imputed = imputer.fit_transform(X_train_encoded)

# Now, train the RandomForestClassifier using X_train_imputed
clf.fit(X_train_imputed, y_train_encoded)

In [ ]:
# Concatenate the training and test sets
concatenated_df = pd.concat([X_train, X_test], axis=0)

In [ ]:
# One-hot encode categorical columns in the concatenated DataFrame
concatenated_encoded = pd.get_dummies(concatenated_df, columns=categorical_columns)

In [ ]:
# Separate the training and test sets after one-hot encoding
X_train_encoded = concatenated_encoded.iloc[:len(X_train)]
X_test_encoded = concatenated_encoded.iloc[len(X_train):]

In [ ]:
# Impute missing values in the test set using the same imputer fitted on the training set
X_test_imputed = imputer.transform(X_test_encoded)

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Instantiate LabelEncoder
label_encoder = LabelEncoder()

# Encode the target variable (ActivityType) into numerical values
y_test_encoded = label_encoder.fit_transform(y_test)

# Now, evaluate the performance of the trained model on the test set
y_pred = clf.predict(X_test_imputed)
print(classification_report(y_test_encoded, y_pred))

In [ ]:
# Now, evaluate the performance of the trained model on the test set
y_pred = clf.predict(X_test_imputed)
print(classification_report(y_test_encoded, y_pred))

In [ ]:
# Concatenate the training and test sets
concatenated_df = pd.concat([X_train, X_test], axis=0)

# One-hot encode categorical columns in the concatenated DataFrame
concatenated_encoded = pd.get_dummies(concatenated_df, columns=categorical_columns)

# Separate the training and test sets after one-hot encoding
X_train_encoded = concatenated_encoded.iloc[:len(X_train)]
X_test_encoded = concatenated_encoded.iloc[len(X_train):]

# Impute missing values in the test set using the same imputer fitted on the training set
X_test_imputed = imputer.transform(X_test_encoded)

# Now, evaluate the performance of the trained model on the test set
y_pred = clf.predict(X_test_imputed)
print(classification_report(y_test_encoded, y_pred))

In [ ]:
# Assuming X_train contains the features for training
# Identify categorical columns
categorical_columns = X_train.select_dtypes(include=['object']).columns

In [ ]:
# Assuming X_train contains the features for training
# Identify categorical columns
categorical_columns = X_train.select_dtypes(include=['object']).columns

In [ ]:

# Now, train the RandomForestClassifier using X_train_encoded
clf.fit(X_train_encoded, y_train_encoded)

In [ ]:


# Assuming X_train contains the features for training
# Identify categorical columns
categorical_columns = X_train.select_dtypes(include=['object']).columns


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Prepare features and labels
X = df_135.drop(['ActivityType', 'Timestamp'], axis=1)
y = df_135['ActivityType']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(X_train, y_train)

# Predict activity type for Unknown data points
unknown_data = df_135[df_135['ActivityType'] == 'Unknown']
unknown_features = unknown_data.drop(['ActivityType', 'Timestamp'], axis=1)
predicted_activity_types = clf.predict(unknown_features)

In [ ]:
unknown_data['on_lift'].value_counts()

In [ ]:
unknown_features['on_lift'].value_counts()

In [ ]:
df_135['on_lift'].value_counts()

In [ ]:
predicted_activity_types

In [ ]:
df_135['Timestamp'] = pd.to_datetime(df_135['Timestamp'])

In [ ]:
# Calculate speed based on consecutive data points
df_135['Speed'] = df_135['Speed(m/s)'].diff() / df_135['Timestamp'].diff().dt.total_seconds()

In [ ]:
df_135['Speed'].value_counts()

In [ ]:
df_135['Speed'].sum

In [ ]:
df_135_unkown = df_135[df_135['ActivityType'] == 'Unknown']

In [ ]:
# Convert 'Timestamp' to datetime if it's not already in datetime format
df_135_unkown['Timestamp'] = pd.to_datetime(df_135_unkown['Timestamp'])

In [ ]:
# Convert 'Timestamp' to datetime if it's not already in datetime format
df_135_unkown['Timestamp'] = pd.to_datetime(df_135_unkown['Timestamp'])

# Sort the DataFrame by the 'Timestamp' column
df_135_unkown = df_135_unkown.sort_values(by='Timestamp')

# Calculate the speed
df_135_unkown['Speed'] = df_135_unkown['Speed(m/s)'].diff() / df_135_unkown['Timestamp'].diff().dt.total_seconds()

# Display the DataFrame
print(df_135_unkown)

In [ ]:
df_135_unkown['Speed'].value_counts()

In [ ]:
df_135_unkown['Speed']

In [ ]:
file_166='/content/df_166m.csv'
df_166=pd.read_csv(file_166)

In [ ]:
file_206='/content/df_206_labeled_on_lift.csv'
df_206=pd.read_csv(file_206)

In [ ]:
file_290='/content/df_290_labeled_on_lift.csv'
df_290=pd.read_csv(file_290)

In [ ]:
file_310='/content/df_310_labeled_on_lift.csv'
df_310=pd.read_csv(file_310)

In [ ]:
file_57='/content/df_57_labeled_on_lift.csv'
df_57=pd.read_csv(file_57)

In [ ]:
file_95='/content/df_95_labeled_on_lift.csv'
df_95=pd.read_csv(file_95)

In [ ]:
df_166

In [ ]:
df_166.info()

In [ ]:
df_166.columns

In [ ]:
import geopandas as gpd
from shapely.geometry import Point

# Coordinate system
crs = {'init': 'epsg:4326'}

# Create GPS points
geometry = [Point(xy) for xy in zip(df_166['Long'], df_166['Lat'])]

# Create a GeoDataFrame with GPS data points
geo_df = gpd.GeoDataFrame(df_166, crs=crs, geometry=geometry)

# Reduce data to every 60 seconds
reduced_df = df_166.iloc[::60, :].copy()

# Update the geometry column in the reduced DataFrame
reduced_df['geometry'] = [Point(xy) for xy in zip(reduced_df['Long'], reduced_df['Lat'])]
reduced_geo_df = gpd.GeoDataFrame(reduced_df, crs=crs, geometry='geometry')

reduced_df.head(5)

In [ ]:
import geopandas as gpd
from shapely.geometry import Point

# Coordinate system
crs = {'init': 'epsg:4326'}

# Function to create GeoDataFrame and reduce data
def create_and_reduce(df):
    # Create GPS points
    geometry = [Point(xy) for xy in zip(df['Long'], df['Lat'])]

    # Create a GeoDataFrame with GPS data points
    geo_df = gpd.GeoDataFrame(df, crs=crs, geometry=geometry)

    # Reduce data to every 60 seconds
    reduced_df = df.iloc[::60, :].copy()

    # Update the geometry column in the reduced DataFrame
    reduced_df['geometry'] = [Point(xy) for xy in zip(reduced_df['Long'], reduced_df['Lat'])]
    reduced_geo_df = gpd.GeoDataFrame(reduced_df, crs=crs, geometry='geometry')

    return reduced_df

# Create and reduce GeoDataFrames for each DataFrame
reduced_df_135 = create_and_reduce(df_135)
reduced_df_206 = create_and_reduce(df_206)
reduced_df_290 = create_and_reduce(df_290)
reduced_df_310 = create_and_reduce(df_310)
reduced_df_57 = create_and_reduce(df_57)
reduced_df_95 = create_and_reduce(df_95)

In [ ]:
import folium

# Create a Folium map centered at the first point of the reduced DataFrame
m = folium.Map(location=[reduced_df['Lat'].iloc[0], reduced_df['Long'].iloc[0]], zoom_start=15)

# Add markers for each data point in the reduced DataFrame
for index, row in reduced_df.iterrows():
    folium.Marker([row['Lat'], row['Long']], popup=f"Timestamp: {row['Timestamp']}").add_to(m)

# Display the map
m

In [ ]:
import folium

# Function to create Folium map for reduced DataFrame
def create_folium_map(reduced_df):
    # Create a Folium map centered at the first point of the reduced DataFrame
    m = folium.Map(location=[reduced_df['Lat'].iloc[0], reduced_df['Long'].iloc[0]], zoom_start=15)

    # Add markers for each data point in the reduced DataFrame
    for index, row in reduced_df.iterrows():
        folium.Marker([row['Lat'], row['Long']], popup=f"Timestamp: {row['Timestamp']}").add_to(m)

    # Display the map
    return m

# Create Folium maps for all reduced DataFrames
folium_maps = {}
for key, reduced_df in {'df_135': reduced_df_135, 'df_206': reduced_df_206, 'df_290': reduced_df_290, 'df_310': reduced_df_310, 'df_57': reduced_df_57, 'df_95': reduced_df_95}.items():
    folium_maps[key] = create_folium_map(reduced_df)

# Display the maps
for key, folium_map in folium_maps.items():
    print(f"Map for {key}:")
    display(folium_map)

In [ ]:
import folium

# Create a Folium map centered at the first point of the reduced DataFrame
m = folium.Map(location=[reduced_df.Lat.iloc[0], reduced_df.Long.iloc[0]], zoom_start=15)

# Add a line to the map
folium.PolyLine(
    list(zip(reduced_df.Lat, reduced_df.Long)),
    color="blue",
    weight=2.5,
    opacity=1
).add_to(m)

# Display the map
m

In [ ]:
import matplotlib.pyplot as plt

# Plot GPS coordinates
plt.figure(figsize=(10, 6))
plt.plot(df_166['Long'], df_166['Lat'], marker='o', linestyle='', markersize=5, color='blue')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('GPS Coordinates')
plt.grid(True)
plt.show()

In [ ]:
import plotly.graph_objects as go

# Create a scatter plot of GPS coordinates
fig = go.Figure(go.Scattermapbox(
    mode="markers",
    lon=df_166['Long'],
    lat=df_166['Lat'],
    marker=go.scattermapbox.Marker(
        size=9,
        color='blue',
        opacity=0.7
    ),
    text=df_166['Timestamp'],  # Use Timestamp as hover text
))

# Set layout parameters for the map
fig.update_layout(
    mapbox=dict(
        accesstoken=YOUR_MAPBOX_ACCESS_TOKEN,  # Replace with your Mapbox access token ,you need an account
        center=dict(
            lon=df_166['Long'].iloc[0],  # Center the map at the first point
            lat=df_166['Lat'].iloc[0]
        ),
        zoom=15
    ),
)

# Show the map
fig.show()

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap

# Create a Basemap instance
plt.figure(figsize=(10, 6))
m = Basemap(projection='merc', llcrnrlat=df_166['Lat'].min(), urcrnrlat=df_166['Lat'].max(),
            llcrnrlon=df_166['Long'].min(), urcrnrlon=df_166['Long'].max(), resolution='i')

# Plot GPS data
m.scatter(df_166['Long'], df_166['Lat'], latlon=True, s=10, c='blue', marker='o', alpha=0.7)

# Draw coastlines, countries, and states
m.drawcoastlines()
m.drawcountries()
m.drawstates()

# Add a title
plt.title('GPS Data')

# Show the plot
plt.show()

In [ ]:
pip install keplergl

In [ ]:
from keplergl import KeplerGl
import pandas as pd

# Create a Kepler.gl map
map_166 = KeplerGl(height=600)
map_166.add_data(data=pd.DataFrame(df_166), name='GPS Data')

# Display the map
map_166

In [ ]:
import folium

# Create a Folium map centered at the first point of the reduced DataFrame
m = folium.Map(location=[reduced_df['Lat'].iloc[0], reduced_df['Long'].iloc[0]], zoom_start=15)

# Add markers for each data point in the reduced DataFrame
for index, row in reduced_df.iterrows():
    folium.Marker([row['Lat'], row['Long']], popup=f"Timestamp: {row['Timestamp']}").add_to(m)

# Save the map to an HTML file
m.save("map.html")

In [ ]:
import gmplot

# Create a Google Maps plot centered at the first point of the reduced DataFrame
gmap = gmplot.GoogleMapPlotter(reduced_df['Lat'].iloc[0], reduced_df['Long'].iloc[0], 15)

# Plot the coordinates on the map
gmap.scatter(reduced_df['Lat'], reduced_df['Long'], '#FF0000', size=40, marker=False)

# Draw the map to an HTML file
gmap.draw("map.html")

gmap

In [ ]:
from IPython.display import IFrame

IFrame(src='map.html', width=700, height=600)